In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
display(train.shape)
display(test.shape)

(891, 12)

(418, 11)

In [6]:
display(train.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [7]:
display(train['Pclass'].unique())
display(train.groupby(['Pclass'])[['Survived']].apply(lambda x: x.count()))

array([3, 1, 2])

,Survived
Pclass,
1,216
2,184
3,491


In [8]:
display(train.groupby(['Sex'])[['Survived']].apply(lambda x: x.count()))

,Survived
Sex,
female,314
male,577


In [9]:
display(train['Age'].describe())
# Define age bins
bins = [0, 18, 30, 40, 50, 60, 70, 80]
labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-80']

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [36]:
train['Age Group'] = pd.cut(train['Age'], bins=bins, labels=labels, right=False)
test['Age Group'] = pd.cut(test['Age'], bins=bins, labels=labels, right=False)
display(train.groupby(['Age Group'])[['Survived']].apply(lambda x: x.count()))
display(test.head())

/tmp/ipykernel_30/864930245.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(train.groupby(['Age Group'])[['Survived']].apply(lambda x: x.count()))


,Survived
Age Group,
0-18,113
19-30,271
31-40,167
41-50,89
51-60,48
61-70,19
71-80,6


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age Group
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,31-40
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,41-50
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,61-70
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,19-30
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,19-30


In [37]:

display(train['Embarked'].unique())
embarked_mapping = {
    'nan': 0,
     np.nan: 0,
    'S': 1,
    'C': 2,
    'Q': 3
}
train_plne = train.copy()
test_plne = test.copy()
train_plne['Embarked'] = train_plne['Embarked'].replace(embarked_mapping)
test_plne['Embarked'] = test_plne['Embarked'].replace(embarked_mapping)
train_median_age = train_plne['Age'].median()
test_median_age = test_plne['Age'].median()

# Impute NaN values in the Age column with the median age
train_plne['Age'].fillna(train_median_age, inplace=True)
test_plne['Age'].fillna(test_median_age, inplace=True)
display(train_plne.head())
display(test_plne.head())

array(['S', 'C', 'Q', nan], dtype=object)

/tmp/ipykernel_30/854410865.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_plne['Embarked'] = train_plne['Embarked'].replace(embarked_mapping)
/tmp/ipykernel_30/854410865.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_plne['Embarked'] = test_plne['Embarked'].replace(embarked_mapping)
/tmp/ipykernel_30/854410865.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method wil

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age Group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,1,19-30
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,2,31-40
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,19-30
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,1,31-40
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,1,31-40


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age Group
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,3,31-40
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,1,41-50
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,3,61-70
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,1,19-30
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,1,19-30


In [38]:
train_plne = pd.get_dummies(train_plne, columns=['Sex','Embarked','Pclass','Age Group'], prefix=['Sex','Embarked','Pclass','Age Group'], dtype=int)
test_plne = pd.get_dummies(test_plne, columns=['Sex','Embarked','Pclass', 'Age Group'], prefix=['Sex','Embarked','Pclass','Age Group'], dtype=int)

In [39]:
train_plne.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_female,...,Pclass_1,Pclass_2,Pclass_3,Age Group_0-18,Age Group_19-30,Age Group_31-40,Age Group_41-50,Age Group_51-60,Age Group_61-70,Age Group_71-80
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0,...,0,0,1,0,1,0,0,0,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1,...,1,0,0,0,0,1,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,...,0,0,1,0,1,0,0,0,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,...,1,0,0,0,0,1,0,0,0,0
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0,...,0,0,1,0,0,1,0,0,0,0


In [40]:
nan_counts = train_plne.isna().sum()
display(nan_counts)
nan_counts = test_plne.isna().sum()
display(nan_counts)

PassengerId          0
Survived             0
Name                 0
Age                  0
SibSp                0
Parch                0
Ticket               0
Fare                 0
Cabin              687
Sex_female           0
Sex_male             0
Embarked_0           0
Embarked_1           0
Embarked_2           0
Embarked_3           0
Pclass_1             0
Pclass_2             0
Pclass_3             0
Age Group_0-18       0
Age Group_19-30      0
Age Group_31-40      0
Age Group_41-50      0
Age Group_51-60      0
Age Group_61-70      0
Age Group_71-80      0
dtype: int64

PassengerId          0
Name                 0
Age                  0
SibSp                0
Parch                0
Ticket               0
Fare                 1
Cabin              327
Sex_female           0
Sex_male             0
Embarked_1           0
Embarked_2           0
Embarked_3           0
Pclass_1             0
Pclass_2             0
Pclass_3             0
Age Group_0-18       0
Age Group_19-30      0
Age Group_31-40      0
Age Group_41-50      0
Age Group_51-60      0
Age Group_61-70      0
Age Group_71-80      0
dtype: int64

In [41]:
train_plne.dtypes

PassengerId          int64
Survived             int64
Name                object
Age                float64
SibSp                int64
Parch                int64
Ticket              object
Fare               float64
Cabin               object
Sex_female           int64
Sex_male             int64
Embarked_0           int64
Embarked_1           int64
Embarked_2           int64
Embarked_3           int64
Pclass_1             int64
Pclass_2             int64
Pclass_3             int64
Age Group_0-18       int64
Age Group_19-30      int64
Age Group_31-40      int64
Age Group_41-50      int64
Age Group_51-60      int64
Age Group_61-70      int64
Age Group_71-80      int64
dtype: object

Check if the age group is a feature that can be used, dummy variable
Cabin seems to have 40% values missing but maybe useful to show likelihood of survival, can be a dummy variable
Maybe PClass should be a dummy variable
We should also look at the fares and see if passengers with higher or lower fare had more probability of survival maybe can be dummy variable based on range.

In [49]:
train_plne.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Sex_female', 'Sex_male', 'Embarked_0', 'Embarked_1',
       'Embarked_2', 'Embarked_3', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Age Group_0-18', 'Age Group_19-30', 'Age Group_31-40',
       'Age Group_41-50', 'Age Group_51-60', 'Age Group_61-70',
       'Age Group_71-80'],
      dtype='object')

In [51]:
drop_columns = ['Name','Age','SibSp','Parch','Fare','Ticket','Cabin']
train_plne.drop(drop_columns, axis=1)

,PassengerId,Survived,Sex_female,Sex_male,Embarked_0,Embarked_1,Embarked_2,Embarked_3,Pclass_1,Pclass_2,Pclass_3,Age Group_0-18,Age Group_19-30,Age Group_31-40,Age Group_41-50,Age Group_51-60,Age Group_61-70,Age Group_71-80
0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0
1,2,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0
2,3,1,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0
3,4,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0
4,5,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0
887,888,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0
888,889,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
889,890,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0


In [116]:
import pandas as pd
train_num = train_plne.drop(drop_columns, axis=1)
test_num = test_plne.drop(drop_columns, axis=1)
X = train_num.drop(['Survived','PassengerId'], axis=1)
y = train_num['Survived'].to_frame()

In [120]:
import torch
xs = torch.Tensor(X.values)
ys = torch.Tensor(y.values)
xs[0]

tensor([0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.])

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0
        self._backward = lambda: None

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
         other = other if isinstance(other, Value) else Value(other)
         out = Value(self.data + other.data, (self, other),'+')
         def _backward():
             self.grad += 1.0 * out.grad
             other.grad += 1.0 * out.grad
         out._backward = _backward
         return out

    def __radd__(self, other): # other + self
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other),'*')
        def _backward():
            self.grad += other.data * out.grad 
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other): # other * self
        return self * other
    
        
    def __pow__(self, other):
        assert isinstance(other,(int, float)), "only supporting int/float powers for now."
        out = Value(self.data**other, (self,),f'**{other}')
        def _backward():
            self.grad += (other * self.data ** (other - 1)) * out.grad # using power rule deriving local derivative
        out._backward = _backward
        return out

    def __truediv__(self, other): #self / other (invert other)
        return self * other**-1

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def tanh(self,):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self,),'tanh')

        def _backward():
            self.grad += ((1 - (t**2)) * out.grad)
        out._backward = _backward
        return out

    def exp(self,):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out

    def backward(self,):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self) 
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [ ]:
import math
import random

class Neuron:
    def __init__(self, nin, nonlin=True):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin = nonlin

    def __call__(self, x):
        act = sum([x*w for x,w in zip(x,self.w)], self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nouts):
        self.neurons = [Neuron(nin) for _ in range(nouts)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs)==1 else outs

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]
        



In [ ]:
row_size = len(list(X[0]))


In [ ]:

n = MLP(row_size,[row_size+1,row_size+1,1])
n(X[0])


In [ ]:
for k in range(30):
    #forward pass
    ypred = [n(x) for x in X]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(y, ypred))

    for p in n.parameters():
        p.grad = 0.0
    # backward pass
    loss.backward()

    # Update
    for p in n.parameters():
        p.data += -0.1 * p.grad

    print(k, loss.data)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
train_num = train_plne[['Pclass','Sex_female','Sex_male','Age','SibSp','Parch','Fare','Embarked']]
test_num = test_plne[['Pclass','Sex_female','Sex_male','Age','SibSp','Parch','Fare','Embarked']]
train_num_standardized = (train_num - train_num.min()) / (train_num.max() - train_num.min())
test_num_standardized = (test_num - test_num.min()) / (test_num.max() - test_num.min())
train_num_standardized['Survived'] = train_plne['Survived']
X = train_num_standardized[['Pclass','Sex_female','Sex_male','Age','SibSp','Parch','Fare','Embarked']].values
y = train_num_standardized['Survived'].values
# Convert to tensors
features = torch.tensor(X, dtype=torch.float32)
predicted = torch.tensor(y, dtype=torch.float32)
display(features)


X = test_num_standardized[['Pclass','Sex_female','Sex_male','Age','SibSp','Parch','Fare','Embarked']].values
test_data = torch.tensor(X, dtype=torch.float32)

In [ ]:
device = "cpu"

print(f"Using {device} device")

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
# Create an instance of the dataset
dataset = CustomDataset(features, predicted)
# Define the split sizes (e.g., 80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
batch_size = 10

# Create data loaders.
training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
x = training_loader.dataset[2][0]
x

In [ ]:
y = training_loader.dataset[2][1]
y

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()     
model = model.to(device)    

In [ ]:
model = NeuralNetwork().to(device)
print(model)

In [ ]:
logits = model(x.to(device))
print(logits)
y_pred = nn.Softmax(dim=0)(logits)
print(f"Predicted class: {y_pred}")

In [ ]:
print('\n\nModel params:')
for param in model.parameters():
    print(param)

In [ ]:
loss_fn = nn.CrossEntropyLoss()  # CrossEntropyLoss includes softmax
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs
        labels = labels.type(torch.FloatTensor)
        
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs.type(torch.FloatTensor))

        # Compute the loss and its gradients
        print(outputs)
        loss = loss_fn(outputs, labels.type(torch.LongTensor))
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 50 == 0:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for X,y in dataloader:
        optimizer.zero_grad()
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(-1))
    

        # Backpropagation
        loss.backward()
        optimizer.step()

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            labels = model(X)
            test_loss += loss_fn(labels, y.unsqueeze(-1))
            predictions = (labels > 0.5).float()
            correct += (predictions == y).sum().item()
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.2f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
per = 0.4
((per > 0.5) == y)

In [ ]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(val_dataloader, model, loss_fn)
print("Done!")

In [ ]:
ids = []
predictions = []
for (X,id) in zip(test_dataloader, test['PassengerId'].values):
    label = model(X.to(device))
    ids.append(id)
    predictions.append((int((label > 0.5).float().sum().item())))

In [ ]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    'PassengerId': ids,
    'Survived': predictions
})

# Save the DataFrame to a CSV file without the default index
df.to_csv('submission.csv', index=False)